In [34]:
#General Crawler

def SimpleSpider(orders,recipe,output):
    import requests
    import lxml.html
    f = open(orders, 'r')
    dump= f.read()
    f.close()
    l_url = dump.split('\n')
    l_out = []
    for url in l_url:
        if url != '':
            response = requests.get(url)
            html = lxml.html.fromstring(response.content)
            l_res = []
            l_res = html.xpath(recipe)
            l_out.extend(l_res)

    csv_out = "\n".join(l_out)
    f = open(output, 'w')
    f.write(csv_out)
    f.close()

In [27]:
#Execute Crawling


orders = 'target.txt'
recipe = '//a[contains(@href, "entry") and (starts-with(@href,"http://") or starts-with(@href,"https://"))]/@href'
output = 'results.csv'

#SimpleSpider(orders,recipe,output)

In [86]:
#for xpath testing - load

import requests
import lxml.html
url = 'https://www.sakunori.tokyo.jp/entry/h/roppongi'
#url = 'http://www.sakunori.tokyo.jp/entry/yamanote.japanese'

response = requests.get(url)
html = lxml.html.fromstring(response.content)


In [ ]:
#for xpath testing - try recipes

#recipe = '//a[contains(@href, "entry") and (starts-with(@href,"http://") or starts-with(@href,"https://"))]/@href'
#recipe = '//*[@class ="entry-content"]/p/descendant::text()'
#recipe = '//*[@class ="entry-content"]/h3/text()'

#recipe = '//*[@class ="entry-title"]//a/text()'
#recipe = '//*[@class ="entry-content"]//a/text()'

#recipe = '//*[@class ="entry-title"]//a/text()'
#recipe = '//*[@class ="entry-content"]/p/text()'
#recipe = '//*[@class ="entry-content"]/h5/text()'
res = html.xpath(recipe)
res






In [221]:


res = []
response = requests.get(url)
html = lxml.html.fromstring(response.content)

title = html.xpath('//*[@class ="entry-title"]//a/text()')
title = title[0][:title[0].find('　')]
title = title.split('：')

h3 = html.xpath('//*[@class ="entry-content"]/h3')
h3_len =len(h3)
for i in range(h3_len):
    h3_text = h3[i].xpath('text()')
    h5 = h3[i].xpath('following-sibling::h5')
    h5_len =len(h5)
    if h5_len > 1:
        for j in range(h5_len):
            h5_text = h5[j].xpath('text()')
            x = h5[j].xpath('following-sibling::p')
            content = x[0].xpath('descendant::text()')
            a = []
            a.extend(title)
            a.extend(h3_text)
            a.extend(h5_text)
            a.extend(content)
            res.append(a)
    else:
        x = h3[i].xpath('following-sibling::p')
        content = x[0].xpath('descendant::text()')
        a = []
        a.extend(title)
        a.extend(h3_text)
        a.extend('-')
        a.extend(content)
        res.append(a)


        

df = pd.DataFrame(res)
df
#df.to_csv('test.csv', header=None,index=None)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,六本木駅,日比谷線,乗車（停車）位置案内,北千住行き,１号車,（出口３・４）,階段・エスカレーター,７号車,（出口１・２）,階段・エスカレーター・ＥＶ,None,None,None,None,None
1,六本木駅,日比谷線,乗車（停車）位置案内,中目黒行き,１号車,（出口３・４）,階段・エスカレーター,５号車,（出口１・２）,階段・エスカレーター,８号車,（出口１・２）,階段・エレベーター,※７両編成もあり,到着番線により乗車位置が少し変わります。
2,六本木駅,日比谷線,乗り換え案内,-,都営大江戸線,None,None,None,None,None,None,None,None,None,None
3,六本木駅,日比谷線,改札・出口案内,-,出口１ｃ：六本木ヒルズ,都営大江戸線乗り換え：８号車側改札より,トイレ設置箇所,北千住行き：８号車付近,中目黒行き：３号車付近,None,None,None,None,None,None
4,六本木駅,日比谷線,東京メトロ公式サイト：六本木駅,-,www.tokyometro.jp,None,None,None,None,None,None,None,None,None,None


In [1]:
#clean up results

import pandas as pd
df = pd.read_csv('results.csv', header=None)
df = df.drop_duplicates()
df = df[df[0].str.contains('//www.sakunori')]
df.describe()
df.to_csv('results.csv', header=False, index=False)
